In [36]:
import pandas as pd
import numpy as np
import os

In [2]:
def clean_raw_data(df):
    df.columns = df.iloc[0]
    df.drop(0, inplace=True)
    df.drop(["Id", "Geography"], axis=1, inplace=True)
    cols = df.columns
    cols = [x for x in cols if "Margin of Error" in x]
    df.drop(cols, axis=1, inplace=True)
    df.rename(columns={col: col.replace("Estimate; Total:", "Total") for col in df.columns}, inplace=True)
    df.rename(columns={col: col.replace("Estimate: - ", "") for col in df.columns}, inplace=True)
    df.rename(columns={col: col.replace("Estimate; ", "") for col in df.columns}, inplace=True)
    df.rename(columns={col: col.replace("Id2", "Id") for col in df.columns}, inplace=True)
    df.sort_values("Id", inplace=True)
    return df

def clean_list_raw_data(dfs):
    cleaned_dfs = []
    for df in dfs:
        cleaned_dfs.append(clean_raw_data(df.copy()))
        
    df_length = [x.shape[0] for x in cleaned_dfs]
    assert len(set(df_length)) == 1
    results = pd.concat(cleaned_dfs, axis=1)
    results = results.loc[:,~results.columns.duplicated()]
    return results

def compile_county_data(folder):
    my_data = []
    for file in os.listdir(folder):
        if file.endswith("with_ann.csv"):
            my_data.append(pd.read_csv(os.path.join(folder, file)))
    results = clean_list_raw_data(my_data)
    results.rename({'Id': "BLOCK"}, axis=1, inplace=True)
    return results

In [3]:
census = compile_county_data("data/census_data")
census.to_csv("data/census_data.csv")

In [4]:
def compile_crime_data(folder):
    my_data = []
    for file in os.listdir(folder):
        if file.startswith("crime_blocks"):
            my_data.append(pd.read_csv(os.path.join(folder, file)))
    results = pd.concat(my_data, axis=0)
    results.drop("Unnamed: 0", axis=1, inplace=True)
    
    crimes = pd.read_csv('data/NYPD_Complaint_Data_Historic.csv')
    assert len(set(crimes["CMPLNT_NUM"])) == len(list(crimes["CMPLNT_NUM"]))
    crimes = crimes[crimes["CMPLNT_NUM"].isin(list(results["CMPLNT_NUM"]))]

    results.sort_values("CMPLNT_NUM", inplace=True)
    crimes.sort_values("CMPLNT_NUM", inplace=True)
    results.reset_index(inplace=True)
    crimes.reset_index(inplace=True)

    results = pd.concat([results, crimes], axis=1)
    results.drop("index", axis=1, inplace=True)
    results = results.loc[:,~results.columns.duplicated()]
    # Dive down to the block group level, not block
    results["BLOCK"] = results["BLOCK"].apply(str).str[:12]
    return results

complaints = compile_crime_data("data")
complaints.to_csv('data/NYPD_Complaint_Data_Historic_with_blocks.csv')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


# How to deal with the crime data?

Decision call
- Group by BLOCK
- Group by day (CMPLNT_FR_DT) or year
- Group by offense level (LAW_CAT_CD)


What to do next?

- One model per day of week


In [54]:
complaints["YEAR"] = complaints['CMPLNT_FR_DT'].str[-4:]

misdemeanor = complaints[complaints["LAW_CAT_CD"] == "MISDEMEANOR"].groupby(["BLOCK", "YEAR"])["CMPLNT_NUM"].count()
felonies = complaints[complaints["LAW_CAT_CD"] == "FELONY"].groupby(["BLOCK", "YEAR"])["CMPLNT_NUM"].count()
violation = complaints[complaints["LAW_CAT_CD"] == "VIOLATION"].groupby(["BLOCK", "YEAR"])["CMPLNT_NUM"].count()

In [55]:
blocks = list(set(complaints["BLOCK"]))
dates = list(set(complaints["YEAR"]))
midx = pd.MultiIndex.from_product([blocks, dates], names=['BLOCK', 'YEAR'])

misdemeanor = misdemeanor.reindex(midx, fill_value=0).reset_index()
felonies = felonies.reindex(midx, fill_value=0).reset_index()
violation = violation.reindex(midx, fill_value=0).reset_index()

In [56]:
full_misdemeanor = misdemeanor.join(census.set_index(census.BLOCK), on="BLOCK", how="left", lsuffix="_complaints")

In [57]:
# Data is very sparse...
print(full_misdemeanor["CMPLNT_NUM"].sum() / full_misdemeanor["CMPLNT_NUM"].count())
full_misdemeanor = full_misdemeanor.drop(["BLOCK_complaints", "YEAR", "BLOCK"], axis=1)
full_misdemeanor.head()

40.6785543703232


,CMPLNT_NUM,Median number of rooms,Total,Total - Imputed,Total - Not imputed,Family households:,Family households: - Married-couple family,Family households: - Other family:,"Family households: - Other family: - Male householder, no wife present","Family households: - Other family: - Female householder, no husband present",...,"Enrolled in school: - Enrolled in college, undergraduate years",Enrolled in school: - Graduate or professional school,Not enrolled in school,"Housing units with a mortgage, contract to purchase, or similar debt:","Housing units with a mortgage, contract to purchase, or similar debt: - With either a second mortgage or home equity loan, but not both:","Housing units with a mortgage, contract to purchase, or similar debt: - With either a second mortgage or home equity loan, but not both: - Second mortgage only","Housing units with a mortgage, contract to purchase, or similar debt: - With either a second mortgage or home equity loan, but not both: - Home equity loan only","Housing units with a mortgage, contract to purchase, or similar debt: - Both second mortgage and home equity loan","Housing units with a mortgage, contract to purchase, or similar debt: - No second mortgage and no home equity loan",Housing units without a mortgage
0,12,4.4,618,7,611,228,219,9,0,9,...,50,12,636,60,12,6,6,0,48,124
1,19,6.1,414,14,400,154,86,68,14,54,...,22,8,460,130,13,0,13,0,117,55
2,35,3.5,582,0,582,345,109,236,24,212,...,57,12,945,0,0,0,0,0,0,0
3,12,5.0,1160,14,1146,526,327,199,100,99,...,62,31,1357,299,31,0,31,0,268,173
4,4,3.1,1121,32,1089,435,328,107,67,40,...,68,18,1256,69,0,0,0,0,69,22


In [51]:
# data.apply(pd.to_numeric)

for col in full_misdemeanor.columns:
    for idx, row in full_misdemeanor.iterrows():
        try:
            float(full_misdemeanor[col].loc[idx])
        except:
            print(col, idx, full_misdemeanor[col].loc[idx])

KeyboardInterrupt: 

In [67]:
# Check if it is the right way to do it!!
full_misdemeanor.replace('-', 0, inplace=True)
full_misdemeanor.replace({',':'', '\+':''}, regex=True, inplace=True)
full_misdemeanor.fillna(0, inplace=True)

In [68]:
full_misdemeanor.to_csv("data/full_misdemeanor.csv")

In [66]:
full_misdemeanor.loc[:, (full_misdemeanor != 0).any(axis=0)].shape, full_misdemeanor.shape

((6281, 158), (6281, 158))